**All work presented in these notebooks is based on data provided by the research group associated with the work presented in Gardner, Zanic, et al. paper titled: Depolymerizing Kinesins Kip3 and MCAK Shape Cellular Microtubule Architecture by Differential Control of Catastrophe (https://www.sciencedirect.com/science/article/pii/S0092867411012876?via%3Dihub).**

# Are labeled and unlabeled tubulin catastrophe times identically distributed?

In the first notebook, we presented the ECDF's for the labeled and unlabeled tubulin to determine whether the labeling affects the time to catastrophe. These empirical CDF's can be used as plug-in estimates for the actual distribution of the data, to aid in the estimation of parameters that describe the distribution and other useful quantities. 

More specifically, in addition to simply plotting the distribution of the catastrophe times, it would be useful to visualize confidence intervals of the catastrophe data. Confidence intervals give an estimate for the behavior that might be observed were the experiment to be repeated. Thus, 95% confidence intervals of the catastrophe data, as will be shown here, will map a range of time distributions for which 95% of replicates of the experiment are likely to fall within. 

Here, the data can be used for boostrapping, or resampling, to simulate experiment replicates. Plotting the ECDF's of these replicates can give confidence intervals for the observed behavior.

We'll start by loading the previously generated ```dataframe```. Using the ```bokeh_catplot``` plotting package, these bootstrap replicates are automatically generated and the 95% confidence interval is plotted.

In [1]:
import numpy as np
import pandas as pd

import scipy.stats as stats

import bokeh.io
import bokeh.plotting
import bokeh_catplot
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#read in the dataframe
df = pd.read_csv('gardner_time_to_catastrophe_df_tidy.csv',)

df.head()

,time to catastrophe (s),labeled
0,40.0,False
1,60.0,False
2,75.0,False
3,80.0,False
4,85.0,False


In [123]:
# plot the ecdf
p = bokeh_catplot.ecdf(
    data=df,
    cats=['labeled'],
    val='time to catastrophe (s)',
    conf_int=True,
    style='staircase'
)

p.legend.location = 'bottom_right'
p.legend.title = 'Labeled?'

bokeh.io.show(p)

There is a large degree of overlap in the 95% confidence intervals. This reinforces the hypothesis that labeling the microtubules does not affect their catastrophic behavior.

## Estimating the mean & median catastrophe time for labeled vs. unlabeled tubulin

Visually, the distributions above appear very close. However, to further investigate whether there are notable differences not obvious in the ECDF, it would be useful to examine the mean and median for each case. In fact, it would be useful to estimate these values with confidence intervals as described above, to give a range of mean & median values for comparison. The reason for choosing to analyze both mean and median is that both give measures of the central tendency of the data. However, if there are outliers the mean may be significantly skewed by these outliers due to the fact that each point added to the mean is given equal weight, while the median is more stable.

This will be done by writing a series of functions to randomly sample points from the data set, then calculate the mean & median for that sample. This is called "bootstrapping", as we are resampling the observed data to simulate a replicate experiment.

In [14]:
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=len(data))

def draw_bs_reps_mean(data, size=1):
    """Draw boostrap replicates of the mean from 1D data set."""
    out = np.empty(size)
    for i in range(size):
        out[i] = np.mean(draw_bs_sample(data))
    return out

def draw_bs_reps_median(data, size=1):
    """Draw boostrap replicates of the median from 1D data set."""
    out = np.empty(size)
    for i in range(size):
        out[i] = np.median(draw_bs_sample(data))
    return out

These functions were written by Justin Bois ("Lesson 7: Introduction to the Bootstrap", BeBi 103, Caltech).

Now to generate the bootstrap sample of the mean for the datasets:

In [15]:
#get the data for the labeled and unlabeled samples
labeled_data   = df.loc[df['labeled'] == True,  'time to catastrophe (s)'].values
unlabeled_data = df.loc[df['labeled'] == False, 'time to catastrophe (s)'].values

# draw 1000 samples of the mean & median
labeled_boot_mean   = draw_bs_reps_mean(labeled_data, size=10000)
unlabeled_boot_mean = draw_bs_reps_mean(unlabeled_data, size=10000)

labeled_boot_median   = draw_bs_reps_median(labeled_data, size=10000)
unlabeled_boot_median = draw_bs_reps_median(unlabeled_data, size=10000)

Now the numpy percentile function can be used to give the 95% confidence intervals for the means. This range of values lies between the 2.5 and 97.5 percentiles.

In [16]:
mean_conf_int_labeled   = np.percentile(labeled_boot_mean, [2.5, 97.5])
mean_conf_int_unlabeled = np.percentile(unlabeled_boot_mean, [2.5, 97.5])

print("95% confidence interval for mean time to catastrophe, labeled:")
print(mean_conf_int_labeled)
print("\n95% confidence interval for mean time to catastrophe, unlabeled:")
print(mean_conf_int_unlabeled)

95% confidence interval for mean time to catastrophe, labeled:
[401.49170616 481.75533175]

95% confidence interval for mean time to catastrophe, unlabeled:
[353.68421053 476.63289474]


In [17]:
median_conf_int_labeled   = np.percentile(labeled_boot_median, [2.5, 97.5])
median_conf_int_unlabeled = np.percentile(unlabeled_boot_median, [2.5, 97.5])

print("95% confidence interval for median time to catastrophe, labeled:")
print(median_conf_int_labeled)
print("\n95% confidence interval for median time to catastrophe, unlabeled:")
print(median_conf_int_unlabeled)

95% confidence interval for median time to catastrophe, labeled:
[325. 410.]

95% confidence interval for median time to catastrophe, unlabeled:
[280. 410.]


The 95% confidence intervals for the mean & median time to catastrophe are quite similar in their upper bounds, although the lower bounds for catastrophe time for unlabeled samples seem to be significantly lower. 

Based on this result, it is difficult to say whether labeling the tubulin with fluorophores affects the time to catastrophe. While the previously generated ECDF's showed many similarities between the two datasets, this metric indicates that there may be differences that are not visible.

## Testing the hypothesis

Now that it is unclear whether the labeled vs. unlabeled tubulin are identically distributed, it will be useful to conduct a test of this theory to further investigate the potential differences. In particular, a Null Hypothesis Significance Test will indicate the probability that the test statistic (here we will test the distribution of the mean, median and variance) observed will be at least as extreme as that of the observed data. 

To test the hypothesis, we will use a permutation test; a common test to determine whether control and non-control samples come from the same distribution. This test works by merging both datasets, shuffling the data, and drawing random samples from the combined data to be labeled "control" and "non-control". If the results of this shuffling and sampling produce test statistics at least as extreme as those of the original data, it is likely that the hypothesis is true. 

There are 211 labeled samples and 95 unlabeled samples. These datasets will be merged into one array. Then, for every simulation, the list will be shuffled and separated into one 211-element array and another array consisting of the remaining 95 elements. 

From these shuffled and resampled arrays, the bootstrapping and estimation of the mean, median, and variance of each array will be conducted by taking 100 bootstrap samples. 

This procedure will be repeated many times to analyze the statistics of a large sample of bootstrapped values.

In [20]:
# store size of labeled, unlabeled datasets
n = len(labeled_data)
m = len(unlabeled_data)

In [23]:
#concatenate lists
combined_list = list(labeled_data) + list(unlabeled_data)

In [28]:
def draw_bs_reps_all(data, size=100):
    '''Performs [size] boostrap estimates of the mean, median, 
    and variance from 1D data set.
    
    Returns:
    --------
    (size x 1) arrays of bootstrap estimates of mean, median, variance.
    '''
    
    #make an Nx3 array where the 3 columns stand for mean, median, and variance
    mean = np.empty((size, 1))
    median = np.empty((size, 1))
    variance = np.empty((size, 1))
    
    for i in range(size):
        
        boot = draw_bs_sample(data)
        
        mean[i]     = np.mean(boot)
        median[i]   = np.median(boot)
        variance[i] = np.var(boot)
        
    return mean, median, variance

In [29]:
def permutation_test(combo_list, len_labeled, len_unlabeled, num_trials, num_boots):
    '''
    For a combo_list which has both labeled (first) and unlabeled (second) trials, 
    this function performs the following tasks:
    1. scramble the combo_list
    2. break it into two sub-lists of lengths len_labeled and len_unlabeled
    3. perform num_boots bootstraps of these sub-lists 
    4. obtain plug-in estimates for desired statistical functionals 
    5. repeat this procedure x num_trials

    ** "_l" indicates labeled and "_u" indicates unlabeled**
    
    '''
    
    #create empty lists of size num_trials to fill with the individual plug-in estimates
    out_list_mean_l   = num_trials * [None]
    out_list_median_l = num_trials * [None]
    out_list_var_l    = num_trials * [None]
    out_list_mean_u   = num_trials * [None]
    out_list_median_u = num_trials * [None]
    out_list_var_u    = num_trials * [None]
    
    #begin the permutation test
    
    
    for i in range(num_trials):
        
        print('Progress (%): ', str(((i+1)/num_trials) * 100) + "\r", end="")
        
        #scramble the input list
        scramble_list = np.random.permutation(combo_list)
        
        # assign labeled, unlabeled
        list_l = scramble_list[0 : len_labeled]
        list_u = scramble_list[len_labeled+1 : len_labeled+len_unlabeled]
        
        #  call the bootstrap function
        mean_l_list, median_l_list, var_l_list = draw_bs_reps_all(list_l, num_boots)
        mean_u_list, median_u_list, var_u_list = draw_bs_reps_all(list_u, num_boots)
        
        #get the mean value of all the bootstrap simulations
        mean_l   = np.mean(mean_l_list)
        median_l = np.mean(median_l_list)
        var_l    = np.mean(var_l_list)
        
        mean_u   = np.mean(mean_u_list)
        median_u = np.mean(median_u_list)
        var_u    = np.mean(var_u_list)
        
        #update all the relevant lists
        out_list_mean_l[i]   = mean_l
        out_list_median_l[i] = median_l
        out_list_var_l[i]    = var_l
        out_list_mean_u[i]   = mean_u
        out_list_median_u[i] = median_u
        out_list_var_u[i]    = var_u
        
    #package everything together
    out_list_l = [out_list_mean_l, out_list_median_l, out_list_var_l]
    out_list_u = [out_list_mean_u, out_list_median_u, out_list_var_u]
    
    return out_list_l, out_list_u
        

In [27]:
#now give me the results of 1000 permutation tests with 100 bootstrap replicates each
stats_l, stats_u = permutation_test(combined_list, n, m, 1000, 100)

#extract the information we care about
means_l_perm   = stats_l[0]
medians_l_perm = stats_l[1]
vars_l_perm    = stats_l[2]
means_u_perm   = stats_u[0]
medians_u_perm = stats_u[1]
vars_u_perm    = stats_u[2]

"stats_l" and "stats_u" above contain 1000 trials of simulated data for the mean, median, and variance of the first 211 and last 95 samples of the scrambled array containing all of the data. Each trial also consisted of 100 bootstrap replicates for each array.

To compute the probability that the mean, median, and variance observed in future replicate experiments will be at least as extreme as those values observed for the experimental data (given that the two datasets are identically distributed), we will calculate p-values.

In calculating p-values we will define three test statistics: the difference in the means of the labeled (first 211) and unlabeled (last 95) sets; the difference in their medians; and the difference in their variances.

The difference in the mean, median, and variance of the actual data will need to be calculated, and, for each trial, compared to see if the simulated test statistic is more extreme than that of the actual data.

As an example, we define a parameter $\theta$ to be the difference between the means of the two distributions, i.e. $\theta = \bar{L} - \bar{U}$, where L is the set of the labeled tubulin data and U is that of the unlabeled tubulin data. Our null hypothesis is that the differences in means is due to other factors or stochasticity, and not due to the labeling of the tubulin. Thus, with scrambling the data, the estimated $\theta$ should be at least as extreme as that of the experimental data. Our data set has $\bar{L} = 441$ and $\bar{U} = 413$, meaning that our experiment suggests that $\theta$ is 28. 

We now have 1000 plug-in estimates for $\theta$ picked out of our combined and scrambled array. Each of these plug-in estimates $\hat{\theta} = \hat{\bar{L}} - \hat{\bar{U}}$ can be compared to the actual experimental $\theta$ of 28. Note that $\hat{\bar{L}}$ and $\hat{\bar{U}}$ indicate the simulated averages of the "labeled" and "unlabeled" data coming from the permutation test. 

The definition of the p-value is the fraction of instances the test statistic $\hat{\theta}$ is at least as extreme as the test statistic from the measured data (28 for the case of the mean). The number of times $\hat{\theta} > 28$, divided by the total number of simulations (1000), gives the p-value. The smaller the p-value, the less likely it is to observe our experimental value of 28 given that the null hypothesis is true. 

In [38]:
#now find the actual values, and the differences between labeled and unlabeled
mean_l_actual      = np.mean(labeled_data)
mean_u_actual      = np.mean(unlabeled_data)
mean_diff_actual   = mean_l_actual - mean_u_actual

median_l_actual    = np.median(labeled_data)
median_u_actual    = np.median(unlabeled_data)
median_diff_actual = median_l_actual - median_u_actual

var_l_actual    = np.var(labeled_data)
var_u_actual    = np.var(unlabeled_data)
var_diff_actual = var_l_actual - var_u_actual

#find the simulated differences
mean_diff_sim   = np.array(means_l_perm) - np.array(means_u_perm)
median_diff_sim = np.array(medians_l_perm) - np.array(medians_u_perm)
var_diff_sim    = np.array(vars_l_perm) - np.array(vars_u_perm)

In [42]:
def p_calc(exp_value, sim_values_list, extreme='>'):
    '''
    Parameters:
    -----------
    exp_value: test statistic calculated from experimental data
    sim_values_list: list of simulated test statistic values
    extreme: can be '>', '<', '<>', or '><' to indicate calculating the p-
    value based on sim >= exp_value, sim <= exp_value, abs(sim) >= exp_value, or 
    abs(sim) <= exp_value.
    
    Returns:
    --------
    p_value: the calculated p-value for the simulated data
    '''
    
    #initialize a counter for the number of times it is more extreme
    p = 0
    
    if extreme == '>':
        for sim in sim_values_list:
            if sim >= exp_value:
                p += 1
    
    elif extreme == '<':
        for sim in sim_values_list:
            if sim <= exp_value:
                p += 1 
                
    elif extreme == '<>':
        for sim in sim_values_list:
            if abs(sim) >= abs(exp_value):
                p += 1 

    elif extreme == "><":
        for sim in sim_values_list:
            if abs(sim) <= abs(exp_value):
                p += 1
                
    p_value = p / len(sim_values_list)
    
    return p_value

In [40]:
p_mean   = p_calc(mean_diff_actual, mean_diff_sim)
p_median = p_calc(median_diff_actual, median_diff_sim)
p_var    = p_calc(var_diff_actual, var_diff_sim)

In [41]:
print("p-value for the difference of the means being greater than", mean_diff_actual)
print(p_mean)

print("\np-value for the difference of the medians being greater than", median_diff_actual)
print(p_median)

print("\np-value for the difference of the variances being greater than", var_diff_actual)
print(p_var)

p-value for the difference of the means being greater than 28.184584684459935
0.244

p-value for the difference of the medians being greater than 5.0
0.459

p-value for the difference of the variances being greater than -5779.815300582399
0.623


Here, the lowest p-value is that of the difference of the means, labeled - unlabeled. This value indicates that approximately 20-25% of the simulated differences of the means were more extreme than the experimental value of 28. Thus, the majority of the simulations did NOT observe a difference in the means of 28 or more, suggesting that a difference of 28 is not common. This might indicate that the labeled and unlabeled data come from different distributions. However, examining the p-value for the differences of the medians and variances, there are higher numbers, suggesting that the distributions may be identical.

Based on the results of this test, it is difficult to reject the hypothesis that these labeled and unlabeled tubulin are identically distributed. While the evidence in favor of the hypothesis is not particularly strong, given the visual similarity of the distributions, it is still a reasonable assumption.

## Estimating the mean by the Central Limit Theorem

Per the Central Limit Theorem, the mean of the data sets should be approximately normally distributed. Thus, by calculating the mean and standard deviations of the datasets, and using these parameters to sample from a normal distribution (with mean = experimental mean, and sigma = experimental sigma), a 95% confidence interval can be calculated for the mean values. This will be used for comparison of methods to the previously calculated means.

In [44]:
def mu_sigma2(data):
    '''Computes the mean (mu) and standard deviation (sigma2) 
    for a data set inputted as a list.'''
    
    # compute the mean
    mu = np.mean(data)
    
    # size of data
    n = len(data)
    
    #initialize a running sum to keep track of the contribution for each (xi - mu)^2
    running_sum = 0
    
    for x_i in data:
        
        square_diff = (x_i - mu) ** 2
        running_sum += square_diff
        
    sigma2 = running_sum / (n * (n-1))
    
    return mu, sigma2

In [46]:
#get the stats
mu_l, sigma_l2 = mu_sigma2(labeled_data)
mu_u, sigma_u2 = mu_sigma2(unlabeled_data)
sigma_l = np.sqrt(sigma_l2)
sigma_u = np.sqrt(sigma_u2)

conf_int_l = stats.norm.interval(0.95, loc=mu_l, scale=sigma_l)
conf_int_u = stats.norm.interval(0.95, loc=mu_u, scale=sigma_u)

print("95% confidence interval for time to catastrophe, labeled:")
print(conf_int_l)
print("\n95% confidence interval for time to catastrophe, unlabeled:")
print(conf_int_u)

95% confidence interval for time to catastrophe, labeled:
(400.74119602394774, 480.68060492391953)

95% confidence interval for time to catastrophe, unlabeled:
(350.83955128865745, 474.21308029028995)


In the previous calculation of the mean values, the confidence intervals were roughly (400, 480) for labeled and (355, 475) for unlabeled. These estimates are roughly the same.

## Revisiting the ECDF

It is sometimes useful to obtain values of the ECDF at specific values of "x". The following function takes in a dataset, and a list of these specific values of "x", and returns the value of the ECDF corresponding to the x-values for plotting and analysis.

The ECDF of an experimental variable y is the fraction of measurements from the data set that are lower than or equal to an individual measurement y (F(y)). 

Given a dataset, an ECDF "function", F(y), can be made by sorting the data list, and storing each value of the data list ("y") with its corresponding F(y) value.

Thus, if given an array of x-values for which values of F(x) are needed, the highest y-value less than x can be found, and its corresponding F(y) value is returned.

This logic is coded in the following function:

In [136]:
def ecdf(x, data):
    '''
    Finds ecdf values corresponding to the values of x.
    based on the data, then returns ecdf(xi) for all xi in the array x
    '''
    
    # storing the size of the data
    n = len(data)
    
    # presize arrays
    F = np.empty([n, 2])
    ecdf_vals = np.empty(len(x))
    
    # create ECDF "function"
    F[:,0] = sorted(data)
    F[:,1] = np.arange(1,n + 1) / n
    
    # work through data, obtaining values of F(x)
    for j, xi in enumerate(x):
            
        #if xi is smaller than anything in the list, ECDF is 0
        if (xi < F[0,0]):
            value = 0

        # if x > largest element in dataset then ECDF is 1
        elif (xi >= F[-1,0]):
            value = 1

        # else find corresponding element
        else: 
            row = 0
            while xi > F[row,0]:
                row += 1

            value = F[row,1]
        
        ecdf_vals[j] = value
    
    return ecdf_vals

To confirm the function is properly coded, it will be plotted for an array of "x-values" with the experimental data overlayed.

In [137]:
#test it out
ecdf_l_highres = ecdf(np.linspace(0, 1800, 1801), labeled_data)
ecdf_l_actual  = ecdf(labeled_data, labeled_data)

#graph it to make sure it works
p1 = bokeh.plotting.figure(
    height=350,
    width=500,
    x_axis_label='time to catastrophe (s)',
    y_axis_label='ECDF',
)
    
p1.circle(
    x=labeled_data,
    y=ecdf_l_actual,
    color='blue',
    size = 4
)
    
p1.circle(
    x=np.linspace(0, 1800, 1801),
    y=ecdf_l_highres,
    color='orange',
    size=1
)

bokeh.io.show(p1)

1.0
1.0


The "function" produces ECDF values that match the actual data. Looks good!

## DKW Inequality confidence intervals

As one final exploration of the confidence intervals for the distribution of catastrophe times, we will employ the Dvoretzky-Kiefer-Wolfowitz Inequality (DKW) inequality.

The DKW Inequality gives upper and lower bounds of the confidence intervals for the ECDF, by bounding the maximum distance between the ECDF and the actual distribution (CDF). Thus, a confidence interval of $100 \cdot (1-\alpha)$ is computed by solving the following equations:

$\epsilon = \sqrt{{1 \over (2n)} log {2 \over alpha}}$

Lower bound: $L(x) = \text{max}(0, \hat{F}(x) - \epsilon)$

Upper bound: $U(x) = \text{min}(1, \hat{F}(x) + \epsilon)$

Where $\alpha$ for a 95% confidence interval is 0.05, and n is the size of the dataset.

The function below returns these upper and lower bounds for a dataset and defined $\alpha$. These values are then plotted for both the labeled and unlabeled tubulin datasets along with the original ECDFs of the experimental data.

In [114]:
def dkw(data, alpha):
    '''
    Returns lower and upper bounds of the confidence intervals
    for the ECDF of given dataset based on the DKW inequality.
    '''
    
    #calculate epsilon
    n = len(data)
    eps = np.sqrt( (1/(2*n)) * np.log(2/alpha) )
    
    #get the empirical cdf
    data_ecdf = ecdf(data, data)
    
    #initialize the lower and empty bound arrays
    data_lower = np.empty_like(data)
    data_upper = np.empty_like(data)
    
    for i, F_xi in enumerate(data_ecdf):
        
        l = max(0, F_xi - eps)
        u = min(1, F_xi + eps)
        
        data_lower[i] = l
        data_upper[i] = u
        
    return data_lower, data_upper

Now to use this function to plot the confidence intervals for the labeled and unlabeled catastrophe times data.

In [115]:
# call functions, obtain ECDF values
lower_l, upper_l = dkw(labeled_data, 0.05)
lower_u, upper_u = dkw(unlabeled_data, 0.05)

ecdf_labeled = ecdf(labeled_data, labeled_data)
ecdf_unlabeled = ecdf(unlabeled_data, unlabeled_data)

In [116]:
p2 = bokeh.plotting.figure(
    height=350,
    width=500,
    x_axis_label='time to catastrophe (s)',
    y_axis_label='ECDF',
)
    
p2.line(
    x=sorted(labeled_data),
    y=sorted(ecdf_labeled),
    color='blue',
    legend='labeled',
    line_width=4
)

p2.circle(
    x=labeled_data,
    y=lower_l,
    color='blue',
    size=1
)

p2.circle(
    x=labeled_data,
    y=upper_l,
    color='blue',
    size=1
)

p2.line(
    x=sorted(unlabeled_data),
    y=sorted(ecdf_unlabeled),
    color='orange',
    legend='unlabeled',
    line_width=4)

p2.circle(
    x=unlabeled_data,
    y=lower_u,
    color='orange',
    size=1
)

p2.circle(
    x=unlabeled_data,
    y=upper_u,
    color='orange',
    size=1
)

p2.legend.location = 'bottom_right'
bokeh.io.show(p2)

As was seen before in calculating the confidence intervals for the means, the labeled tubulin data are more tightly distributed than those of the unlabeled. However, unlike the previously generated ECDF with confidence intervals, The 95% confidence region for the labeled tubulin lies entirely within the unlabeled region. It appears more likely that these datasets are from the same distribution. 

For this method, part of the reason for the larger confidence intervals in the unlabeled case may be due to the fact that the dataset is less than half the size of the unlabeled ($n = 95$ vs. $n = 211$). This results in a larger epsilon value, and may result in the wider confidence region.